In [11]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from plyfile import PlyData, PlyElement 


In [2]:
import numpy as np
import json
import pandas as pd
import os, glob, re, shutil
import PIL.ExifTags as ExifTags
import PIL.Image as Image
from PIL import ImageOps
import numpy as np
from tqdm import tqdm
from typing import Union
# import cv2

In [ ]:
# same with .npy file just no "data" key
file_path = '/cs/student/projects4/ml/2023/asrivast/datasets/bicycle/poses_bounds.npy'
# load the .npy data
data = np.load(file_path)
print(data.shape)

for i in range(0, 10):
    print(data[i])

# Reading the .ply file from COLMAP and converting it to .npz

format of numpy array to be stored in .npz file
* (N, 7) where N is total number of points
* 0-2 x,y,z
* 3-5 R,G,B
* 7 all ones


Additionally we may remove points outside some bounds

In [ ]:

parent_dir= "/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/"


In [30]:

ply_file= f"{parent_dir}/points3D.ply"
with open(ply_file, 'rb') as f:
    ply_data: PlyData = PlyData.read(f)
    x = ply_data['vertex']['x']
    y = ply_data['vertex']['y']
    
    # make a plot of x and y values using plotly
    z = ply_data['vertex']['z']



    r = ply_data['vertex']['red']
    g = ply_data['vertex']['green']
    b = ply_data['vertex']['blue']
    seven= np.ones(x.shape[0], dtype=np.float32)

    # print data types of each
    print(f"x: {x.dtype}, y: {y.dtype}, z: {z.dtype}, r: {r.dtype}, g: {g.dtype}, b: {b.dtype}, seven: {seven.dtype}")

    # need to change the dtype of R G B to float32 as well
    r = r.astype(np.float32)/255
    g = g.astype(np.float32)/255
    b = b.astype(np.float32)/255


    print(f"x: {x.dtype}, y: {y.dtype}, z: {z.dtype}, r: {r.dtype}, g: {g.dtype}, b: {b.dtype}, seven: {seven.dtype}")

    # filter points based on position
    index= (x<.4) & (x>-.2) & (z<.2) & (z>-.2) 
    x= x[index]
    y= y[index]
    z= z[index]
    r= r[index]
    g= g[index]
    b= b[index]
    seven= seven[index]

    # stack them together
    points = np.vstack((x, y, z, r, g, b, seven)).T
    print(f"points shape: {points.shape}")
    
    # now save it as a .npz file with a key "data"
    npz_file= f"{parent_dir}/init_pt_cld.npz"
    npz= {}
    np.savez_compressed(npz_file, data=points)
    print(f"Saved {npz_file}")


x: float32, y: float32, z: float32, r: uint8, g: uint8, b: uint8, seven: float32
x: float32, y: float32, z: float32, r: float32, g: float32, b: float32, seven: float32
points shape: (5340, 7)
Saved /cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0//init_pt_cld.npz


# Reading points3D.bin and convert it to .npz

In [33]:
import utils_data_making
import utils_colmap

In [36]:
bin_path = os.path.join(f"{parent_dir}/points3D.bin")
xyzs, rgbs, _ = utils_colmap.read_points3D_binary(bin_path)
seg = np.ones_like(xyzs[:, 0])[:, None]   # Always dynamic for now, segmentation always 1
# print current max rgb
print(f"Max RGB: {rgbs.max()}")
rgbs = rgbs / 255.0
print(f"Max RGB: {rgbs.max()}")
pt_cld = dict()
pt_cld = np.concatenate((xyzs, rgbs, seg), axis=1).tolist()
# if not os.path.exists(os.path.join(".")):
#     os.makedirs(os.path.join(args.output_path, args.dataset_name))
np.savez(os.path.join(f'{parent_dir}/init_pt_cld_other.npz'), data=pt_cld)
print('Point cloud saved')

Max RGB: 255.0
Max RGB: 1.0
Point cloud saved


# Reading the .npz file and converting it to .ply file

only for visualisation before we feed it to the Dynamic 3DGS pipeline

In [31]:

# Assuming 'points' is your numpy array of shape (N, 7)
# Load or define your numpy array here if needed
npz_file = f'{parent_dir}/init_pt_cld.npz'

# Load the .npz file
data = np.load(npz_file)
print(data['data'].shape,)
print(data['data'][:,0].dtype, data['data'][:,3].dtype, data['data'][:,6].dtype)
points = data['data']

# Extract the first 6 columns for x, y, z, R, G, B
vertices_data = points[:, :6]

# # Convert to a list of tuples, each tuple representing a vertex
vertices_list = [tuple(vertex) for vertex in vertices_data]
# scale and convert to uint all RGB values
RGB_SCALE = 1 if np.max(vertices_data[:,3:6]) > 1 else 255
print(f"RGB_SCALE: {RGB_SCALE}")
vertices_list = [(x, y, z, int(r* RGB_SCALE), int(g* RGB_SCALE), int(b*RGB_SCALE)) for x, y, z, r, g, b in vertices_list]

# # Define the vertex element
vertex_element = PlyElement.describe(np.array(vertices_list, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')]), 'vertex')

# # Create the PlyData object
ply_data = PlyData([vertex_element])

# # Save the .ply file
ply_file_path = f"{parent_dir}/init_pt_cld.ply"
ply_data.write(ply_file_path)

print(f"PLY file saved to {ply_file_path}")

(5340, 7)
float32 float32 float32
RGB_SCALE: 255
PLY file saved to /cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0//init_pt_cld.ply


# Reading the .bin files from COLMAP and converting it to train_meta.json

## First we see what kind of file is train_meta.json

In [6]:
a= [1,2,3,4,5,6,7]
# get third and fifth element
b= [a[2], a[4]]
b

[3, 5]

In [38]:
parent_dir= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/"

In [66]:
meta_json= f"{parent_dir}/portsmouth/train_meta.json"

k_unique= []
w2c_unique= []
cam_id_unique= []

unique_cam_ids= []

with open(meta_json, 'r') as f:
    meta_dict= json.load(f)
    print(f"Loaded {meta_json}")
    for key in meta_dict.keys():
        value= np.array(meta_dict[key])
        if value.shape == ():
            print(f"{key}: {value}")
        else:
            print(f"{key}: {value.shape}")


        if key == "k":
            total_frames= np.array(meta_dict[key]).shape[0]
            total_cams= np.array(meta_dict[key]).shape[1]
            for frame in range(total_frames):
                for cam in range(total_cams):
                    k_new= meta_dict[key][frame][cam]
                    
                    for old_k in k_unique:
                        if np.allclose(k_new, old_k):
                            break
                    else:
                        k_unique.append(k_new)

        if key == "w2c":
            total_frames= np.array(meta_dict[key]).shape[0]
            total_cams= np.array(meta_dict[key]).shape[1]
            for frame in range(total_frames):
                for cam in range(total_cams):
                    w2c_new= meta_dict[key][frame][cam]
                    
                    for old_w2c in w2c_unique:
                        if np.allclose(w2c_new, old_w2c):
                            break
                    else:
                        w2c_unique.append(w2c_new)

        if key == "cam_id":
            total_frames= np.array(meta_dict[key]).shape[0]
            total_cams= np.array(meta_dict[key]).shape[1]
            for frame in range(total_frames):
                for cam in range(total_cams):
                    cam_id_new= meta_dict[key][frame][cam]
                    
                    for old_cam_id in cam_id_unique:
                        if cam_id_new == old_cam_id:
                            break
                    else:
                        cam_id_unique.append(cam_id_new)

                    

    # print("#####################")
    # k_np= np.array(meta_dict['k'])
    # k_np= k_np[:,(0, -1), :,:]
    # print(k_np.shape)

    # w2c_np= np.array(meta_dict['w2c'])
    # w2c_np= w2c_np[:,(0, -1), :,:]
    # print(w2c_np.shape)

    # fn_np= np.array(meta_dict['fn'])
    # fn_np= fn_np[:,(0, -1)]
    # print(fn_np.shape)

    # cam_id_np= np.array(meta_dict['cam_id'])
    # cam_id_np= cam_id_np[:,(0, -1)]
    # print(cam_id_np.shape)

    # new_meta_dict= meta_dict.copy()
    # new_meta_dict['k']= k_np.tolist()
    # new_meta_dict['w2c']= w2c_np.tolist()
    # new_meta_dict['fn']= fn_np.tolist()
    # new_meta_dict['cam_id']= cam_id_np.tolist()
    # with open("test_meta.json", 'w') as f:
    #     json.dump(new_meta_dict, f, indent=4)
    #     print(f"Saved test_meta.json")

Loaded /cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data//portsmouth/train_meta.json
w: 1220
h: 994
k: (199, 28, 3, 3)
w2c: (199, 28, 4, 4)
fn: (199, 28)
cam_id: (199, 28)


In [67]:
print(len(k_unique))
print(len(w2c_unique))
print(len(cam_id_unique))

1
28
28


# from .bin file

In [68]:
import read_write_model

cams= read_write_model.read_cameras_binary("/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/cameras.bin")
print(cams)
print(type(cams))
print(type(cams[1]))
print(cams[1])

{1: Camera(id=1, model='PINHOLE', width=1220, height=994, params=array([1928.96714337, 1920.45748488,  610.        ,  497.        ]))}
<class 'dict'>
<class 'read_write_model.Camera'>
Camera(id=1, model='PINHOLE', width=1220, height=994, params=array([1928.96714337, 1920.45748488,  610.        ,  497.        ]))


In [69]:
images= read_write_model.read_images_binary("/cs/student/projects4/ml/2023/asrivast/datasets/portsmouth_video/frames/sparse_000001/0/images.bin")
# total number of keys= total number of images
cam_id_map = {} # cameran_num : COLMAP index
for COLMAP_index in images.keys():
    camera_num= int(re.split("[-_]", images[COLMAP_index].name)[1])
    cam_id_map[camera_num]= COLMAP_index
    print("COLMAP index", COLMAP_index, images[COLMAP_index].name, "CAM num", camera_num)

COLMAP index 1 cam-005_000001.jpg CAM num 5
COLMAP index 2 cam-002_000001.jpg CAM num 2
COLMAP index 3 cam-006_000001.jpg CAM num 6
COLMAP index 4 cam-007_000001.jpg CAM num 7
COLMAP index 5 cam-001_000001.jpg CAM num 1
COLMAP index 6 cam-003_000001.jpg CAM num 3
COLMAP index 7 cam-008_000001.jpg CAM num 8
COLMAP index 8 cam-011_000001.jpg CAM num 11
COLMAP index 9 cam-010_000001.jpg CAM num 10
COLMAP index 10 cam-009_000001.jpg CAM num 9
COLMAP index 11 cam-004_000001.jpg CAM num 4
COLMAP index 12 cam-012_000001.jpg CAM num 12
COLMAP index 13 cam-013_000001.jpg CAM num 13
COLMAP index 14 cam-014_000001.jpg CAM num 14
COLMAP index 15 cam-015_000001.jpg CAM num 15
COLMAP index 16 cam-021_000001.jpg CAM num 21
COLMAP index 17 cam-022_000001.jpg CAM num 22
COLMAP index 18 cam-016_000001.jpg CAM num 16
COLMAP index 19 cam-017_000001.jpg CAM num 17
COLMAP index 20 cam-018_000001.jpg CAM num 18
COLMAP index 21 cam-019_000001.jpg CAM num 19
COLMAP index 22 cam-020_000001.jpg CAM num 20
COLMAP

In [117]:
train_cams= [i for i in range(1, 33)] # all cameras
train_cams= [cam_id_map[cam] for cam in train_cams]


test_cams= [2,15,25,30] 
test_cams= [cam_id_map[cam] for cam in test_cams]

# remove test cams from train cams
train_cams= [cam for cam in train_cams if cam not in test_cams]


k_np= np.zeros((len(train_cams),3,3))
w2c_np= np.zeros((len(train_cams),4,4))
fn = []
cam_ids_np= np.zeros(len(train_cams), dtype=np.uint)


for i, image_id in enumerate(train_cams):
    
    img_name= images[image_id].name
    cam_obj= cams[images[image_id].camera_id]

    q_vec= images[image_id].qvec
    t_vec= images[image_id].tvec

    w= cam_obj.width
    h= cam_obj.height

    fx= cam_obj.params[0]
    fy= cam_obj.params[1]
    cx= cam_obj.params[2]
    cy= cam_obj.params[3]


    k= np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    w2c= np.eye(4)
    w2c[:3,:3]= read_write_model.qvec2rotmat(q_vec)
    w2c[:3,3]= t_vec

    k_np[i]= k
    w2c_np[i]= np.linalg.pinv(w2c)
    

    fn.append(img_name)
    cam_ids_np[i]= image_id

# for image_obj in images:
#     print(type(images[image_obj]))

In [119]:
# print shapes of all arrays to which we added in previous steps
print("k ", k_np.shape)
print("w2c ", w2c_np.shape)
print("fn ", len(fn))
print("cam_id", cam_ids_np.shape)

# print

k  (28, 3, 3)
w2c  (28, 4, 4)
fn  28
cam_id (28,)


In [120]:
fn

['cam-001_000001.jpg',
 'cam-003_000001.jpg',
 'cam-004_000001.jpg',
 'cam-005_000001.jpg',
 'cam-006_000001.jpg',
 'cam-007_000001.jpg',
 'cam-008_000001.jpg',
 'cam-009_000001.jpg',
 'cam-010_000001.jpg',
 'cam-011_000001.jpg',
 'cam-012_000001.jpg',
 'cam-013_000001.jpg',
 'cam-014_000001.jpg',
 'cam-016_000001.jpg',
 'cam-017_000001.jpg',
 'cam-018_000001.jpg',
 'cam-019_000001.jpg',
 'cam-020_000001.jpg',
 'cam-021_000001.jpg',
 'cam-022_000001.jpg',
 'cam-023_000001.jpg',
 'cam-024_000001.jpg',
 'cam-026_000001.jpg',
 'cam-027_000001.jpg',
 'cam-028_000001.jpg',
 'cam-029_000001.jpg',
 'cam-031_000001.jpg',
 'cam-032_000001.jpg']

In [121]:
# now add a dimension in beggining in each of them and repeat them total_frames times
total_frames= 199
k_np_full = np.repeat(k_np[np.newaxis, ...], total_frames, axis=0)
w2c_np_full = np.repeat(w2c_np[np.newaxis, ...], total_frames, axis=0)


fn_full= []
for i in range(1, total_frames+1):
    frame_num= str(i).zfill(6)
    fn_new= fn.copy()
    for i,_ in enumerate(fn_new):
        cam_num= int(re.split("[-_]", _)[1])
        fn_new[i]= fn_new[i].replace("000001", frame_num)
        fn_new[i]= f"{cam_num}/{fn_new[i][8:-4]}.JPG"

        
    fn_full.append(fn_new)

cam_ids_np_full= np.repeat(cam_ids_np[np.newaxis, ...], total_frames, axis=0)

In [122]:
# print shapes along with names agains
print("k ", k_np_full.shape)
print("w2c ", w2c_np_full.shape)
print("fn ", np.array(fn_full).shape)
print("cam_id", cam_ids_np_full.shape)


k  (199, 28, 3, 3)
w2c  (199, 28, 4, 4)
fn  (199, 28)
cam_id (199, 28)


In [123]:
fn_full[-1]

['1/000199.JPG',
 '3/000199.JPG',
 '4/000199.JPG',
 '5/000199.JPG',
 '6/000199.JPG',
 '7/000199.JPG',
 '8/000199.JPG',
 '9/000199.JPG',
 '10/000199.JPG',
 '11/000199.JPG',
 '12/000199.JPG',
 '13/000199.JPG',
 '14/000199.JPG',
 '16/000199.JPG',
 '17/000199.JPG',
 '18/000199.JPG',
 '19/000199.JPG',
 '20/000199.JPG',
 '21/000199.JPG',
 '22/000199.JPG',
 '23/000199.JPG',
 '24/000199.JPG',
 '26/000199.JPG',
 '27/000199.JPG',
 '28/000199.JPG',
 '29/000199.JPG',
 '31/000199.JPG',
 '32/000199.JPG']

In [125]:
meta_dict= {"w": 1220,
            "h": 994,
            "k": k_np_full.tolist(),
            "w2c": w2c_np_full.tolist(),
            "fn": fn_full,
            # "cam_id": cam_ids_np_full.tolist()
            }

json_path= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/portsmouth/train_meta.json"
with open(json_path, 'w') as meta_json:
    json.dump(meta_dict, meta_json, indent=4)
    print(f"Saved {json_path}")


Saved /cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/portsmouth/train_meta.json


# Reading the output .npz file

In [65]:
npz_file= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/output/exp3/boxes/params.npz"
with np.load(npz_file) as data:
    for file in data.files:
        print(f"{file}: {data[file].shape}")

        if file == "cam_m" or file == "cam_c":
            print(data[file])

    

means3D: (11, 355146, 3)
rgb_colors: (11, 355146, 3)
seg_colors: (355146, 3)
unnorm_rotations: (11, 355146, 4)
logit_opacities: (355146, 1)
log_scales: (355146, 3)
cam_m: (50, 3)
[[ 0.04052375  0.04089984  0.03511785]
 [-0.10412157 -0.11086568 -0.11600678]
 [ 0.03189565  0.04502699  0.03034953]
 [-0.06717889 -0.07117973 -0.08319283]
 [-0.00327357  0.01511976  0.0076265 ]
 [ 0.09751539  0.09298394  0.10647158]
 [ 0.05988395  0.03975919  0.05563907]
 [ 0.00337028  0.01964025  0.00806313]
 [-0.00923424  0.01268488  0.0058958 ]
 [-0.11130295 -0.11080908 -0.11390278]
 [-0.01869849 -0.02034263 -0.01551975]
 [-0.05780772 -0.06925236 -0.06072897]
 [-0.01703494 -0.00727891 -0.01703449]
 [-0.0618599  -0.06228618 -0.06967499]
 [ 0.02166728  0.01342456  0.01806911]
 [-0.02762344 -0.01596763 -0.02078595]
 [ 0.06977711  0.07378475  0.07342494]
 [ 0.00482329  0.03206026  0.01437046]
 [-0.06484304 -0.0602063  -0.05086702]
 [-0.01094176  0.00574239 -0.00309625]
 [-0.01958307 -0.01359787 -0.03105757]
 [

# Move first frame to a new folder

In [22]:
import pandas as pd
import os, glob, re, shutil
import PIL.ExifTags as ExifTags
import PIL.Image as Image
from PIL import ImageOps
import numpy as np
from tqdm import tqdm
from typing import Union
import piexif

In [27]:
all_jpgs= glob.glob("/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/basketball/ims/*/000000.jpg")
print(len(all_jpgs))

new_folder= "/cs/student/projects4/ml/2023/asrivast/datasets/dynamic_3DGS/data/basketball/frame_000000"
os.makedirs(new_folder, exist_ok=True)
for jpg in tqdm(all_jpgs):
    cam_num= os.path.basename(os.path.dirname(jpg))
    frame0_file= os.path.join(new_folder, f"cam-{cam_num}_000000.jpg")
    shutil.copy2(jpg, frame0_file)

31


100%|██████████| 31/31 [00:00<00:00, 527.43it/s]
